In [2]:
class Lexer:
    def __init__(self, input_string):
        self.tokens = self.tokenize(input_string)
        self.current_index = 0

    def tokenize(self, input_string):
        terminal_symbols = [",", ")", "(", "true", "false", "*", "/", "+", "-", "or", "and", "not", "end", "const", "def", "=", "<", ">"]
        words = input_string.split()
        tokens = []

        for i, word in enumerate(words):
            if word.isdigit():
                tokens.append("const")
            elif word not in terminal_symbols:
                tokens.append("def" if (i + 1 < len(words) and words[i + 1] == "(") else "var")
            else:
                tokens.append(word)

        tokens.append("end")
        return tokens

    def get_current_token(self):
        if self.current_index < len(self.tokens):
            return self.tokens[self.current_index]
        return None

    def accept(self, symbol):
        current_token = self.get_current_token()
        if current_token == symbol:
            self.current_index += 1
            return True
        return False

    def error(self, symbol, valid_symbols):
        if not self.check(symbol, valid_symbols):
            raise ValueError("ERROR")

    def check(self, symbol, valid_symbols):
      return symbol in valid_symbols

In [3]:
class Stack:
    def __init__(self):
        self.stack = []

    def push(self, value):
        self.stack.append(value)

    def pop(self):
        if not self.is_empty():
            return self.stack.pop()
        return None

    def is_empty(self):
        return len(self.stack) == 0

    def __repr__(self):
        return str(self.stack)


In [4]:
class IState:
    def execute(self):
      pass

class ZeroState(IState):
    def __init__(self, current_state, symbol, next_state, stack, lexer, valid_tokens):
        self.current_state = current_state
        self.symbol = symbol
        self.next_state = next_state
        self.stack = stack
        self.lexer = lexer

    def execute(self):
        return self.next_state
    
class ErrorState(IState):
    def __init__(self, current_state, symbol, next_state, stack, lexer, valid_tokens):
        self.current_state = current_state
        self.symbol = symbol
        self.next_state = next_state
        self.stack = stack
        self.lexer = lexer
        self.valid_tokens = valid_tokens

    def execute(self):
        self.lexer.error(self.symbol, self.valid_tokens)
        return self.next_state

class StackErrorState(IState):
    def __init__(self, current_state, symbol, next_state, stack, lexer, valid_tokens):
        self.current_state = current_state
        self.symbol = symbol
        self.next_state = next_state
        self.stack = stack
        self.lexer = lexer
        self.valid_tokens = valid_tokens

    def execute(self):
        self.lexer.error(self.symbol, self.valid_tokens)
        print(f"Pushing to stack: {self.current_state}")
        self.stack.push(self.current_state + 1)
        return self.next_state


class ReturnErrorState(IState):
    def __init__(self, current_state, symbol, next_state, stack, lexer, valid_tokens):
        self.current_state = current_state
        self.symbol = symbol
        self.next_state = next_state
        self.stack = stack
        self.lexer = lexer
        self.valid_tokens = valid_tokens

    def execute(self):
        self.lexer.error(self.symbol, self.valid_tokens)
        stack_value = self.stack.pop() if not self.stack.is_empty() else self.next_state
        print(f"Popping from stack: {stack_value}")
        return stack_value
    
class AcceptErrorState(IState):
    def __init__(self, current_state, symbol, next_state, stack, lexer, valid_tokens):
        self.current_state = current_state
        self.symbol = symbol
        self.next_state = next_state
        self.stack = stack
        self.lexer = lexer
        self.valid_tokens = valid_tokens

    def execute(self):
        self.lexer.error(self.symbol, self.valid_tokens)
        print(f"ACCEPTING: {self.symbol}")
        self.lexer.accept(self.symbol)
        return self.next_state


class ReturnAcceptErrorState(IState):
    def __init__(self, current_state, symbol, next_state, stack, lexer, valid_tokens):
        self.current_state = current_state
        self.symbol = symbol
        self.next_state = next_state
        self.stack = stack
        self.lexer = lexer
        self.valid_tokens = valid_tokens

    def execute(self):
        self.lexer.error(self.symbol, self.valid_tokens)
        print(f"ACCEPTING: {self.symbol}")
        self.lexer.accept(self.symbol)
        return self.stack.pop() if not self.stack.is_empty() else self.next_state


In [5]:
class LL1Parser:
    def __init__(self, transitions):
        self.transitions = transitions
        self.state_map = {
            (0, 0, 0, 0): ZeroState,
            (0, 0, 0, 1): ErrorState,
            (0, 1, 0, 1): StackErrorState,
            (0, 0, 1, 1): AcceptErrorState,
            (1, 0, 0, 1): ReturnErrorState,
            (1, 0, 1, 1): ReturnAcceptErrorState
        }
        self.current_state = 1
        self.stack = Stack()
        self.lexer = None

    def process(self, tokens):
        self.lexer = Lexer(" ".join(tokens))
        current_token = self.lexer.get_current_token()
        
        while current_token is not None:
            if current_token == 'end' and self.stack.is_empty():
              print("Parsing complete")
              break

            state_data = self.transitions[self.current_state]
            print(f"State: {self.current_state}\t Token: {current_token}\t Stack: {self.stack}")

            valid_tokens, next_state, attr1, attr2, attr3, attr4 = state_data
            if not self.lexer.check(current_token, valid_tokens):
                if attr4 == 0:
                    print(f"Unexpected symbol '{current_token}', skipping to state {self.current_state + 1}")
                    self.current_state += 1
                    current_token = self.lexer.get_current_token()
                    continue

            state_key = (attr1, attr2, attr3, attr4)
            state_class = self.state_map[state_key]
            state_instance = state_class(self.current_state, current_token, next_state, self.stack, self.lexer, valid_tokens)
            self.current_state = state_instance.execute()
            current_token = self.lexer.get_current_token()

        print("Grammar is true")


In [6]:
import pandas as pd

df = pd.read_excel("РИС_ЛР_2.xlsx", index_col = "№")
df

,множество направляющих символов,next,return,stack,accept,error
№,,,,,,
1,end ) ( const var true false not def,29,0,0,0,1
2,+,31,0,0,0,0
3,-,34,0,0,0,0
4,end ),37,0,0,0,1
5,( def var const true false not,38,0,0,0,1
...,...,...,...,...,...,...
85,",",86,0,0,1,1
86,end ) ( const var true false not def,28,0,1,0,1
87,end ) ( const var true false not def,26,0,0,0,1


In [7]:
dictionary = {}
for i in range(1, df.shape[0] + 1):
  dictionary[i] = []
  for j in df.columns:
    dictionary[i].append(df[j][i])

dictionary

{1: ['end ) ( const var  true false not def',
  np.int64(29),
  np.int64(0),
  np.int64(0),
  np.int64(0),
  np.int64(1)],
 2: ['+', np.int64(31), np.int64(0), np.int64(0), np.int64(0), np.int64(0)],
 3: ['-', np.int64(34), np.int64(0), np.int64(0), np.int64(0), np.int64(0)],
 4: ['end )',
  np.int64(37),
  np.int64(0),
  np.int64(0),
  np.int64(0),
  np.int64(1)],
 5: ['( def var const true false not',
  np.int64(38),
  np.int64(0),
  np.int64(0),
  np.int64(0),
  np.int64(1)],
 6: ['*', np.int64(40), np.int64(0), np.int64(0), np.int64(0), np.int64(0)],
 7: ['/', np.int64(43), np.int64(0), np.int64(0), np.int64(0), np.int64(0)],
 8: ['end ) + -',
  np.int64(46),
  np.int64(0),
  np.int64(0),
  np.int64(0),
  np.int64(1)],
 9: ['( def var const true false not',
  np.int64(47),
  np.int64(0),
  np.int64(0),
  np.int64(0),
  np.int64(1)],
 10: [',', np.int64(49), np.int64(0), np.int64(0), np.int64(0), np.int64(0)],
 11: ['<', np.int64(52), np.int64(0), np.int64(0), np.int64(0), np.int64(

In [8]:
input = "27 - 10 * ( 10 + 6 / 7 - ( 30 - 2 ) )"
lexer = Lexer(input)
tokens = lexer.tokens
ll1 = LL1Parser(dictionary)
print(f"\nINPUT: {input}")
print("----------------------------")
ll1.process(tokens)



INPUT: 27 - 10 * ( 10 + 6 / 7 - ( 30 - 2 ) )
----------------------------
State: 1	 Token: const	 Stack: []
State: 29	 Token: const	 Stack: []
Pushing to stack: 29
State: 5	 Token: const	 Stack: [np.int64(30)]
State: 38	 Token: const	 Stack: [np.int64(30)]
Pushing to stack: 38
State: 9	 Token: const	 Stack: [np.int64(30), np.int64(39)]
State: 47	 Token: const	 Stack: [np.int64(30), np.int64(39)]
Pushing to stack: 47
State: 15	 Token: const	 Stack: [np.int64(30), np.int64(39), np.int64(48)]
Unexpected symbol 'const', skipping to state 16
State: 16	 Token: const	 Stack: [np.int64(30), np.int64(39), np.int64(48)]
Unexpected symbol 'const', skipping to state 17
State: 17	 Token: const	 Stack: [np.int64(30), np.int64(39), np.int64(48)]
Unexpected symbol 'const', skipping to state 18
State: 18	 Token: const	 Stack: [np.int64(30), np.int64(39), np.int64(48)]
State: 70	 Token: const	 Stack: [np.int64(30), np.int64(39), np.int64(48)]
ACCEPTING: const
State: 48	 Token: -	 Stack: [np.int64(30), 

In [10]:
input = "20 * 2 /"
lexer = Lexer(input)
tokens = lexer.tokens
ll1 = LL1Parser(dictionary)
print(f"\nINPUT: {input}")
print("----------------------------")
ll1.process(tokens)


INPUT: 20 * 2 /
----------------------------
State: 1	 Token: const	 Stack: []
State: 29	 Token: const	 Stack: []
Pushing to stack: 29
State: 5	 Token: const	 Stack: [np.int64(30)]
State: 38	 Token: const	 Stack: [np.int64(30)]
Pushing to stack: 38
State: 9	 Token: const	 Stack: [np.int64(30), np.int64(39)]
State: 47	 Token: const	 Stack: [np.int64(30), np.int64(39)]
Pushing to stack: 47
State: 15	 Token: const	 Stack: [np.int64(30), np.int64(39), np.int64(48)]
Unexpected symbol 'const', skipping to state 16
State: 16	 Token: const	 Stack: [np.int64(30), np.int64(39), np.int64(48)]
Unexpected symbol 'const', skipping to state 17
State: 17	 Token: const	 Stack: [np.int64(30), np.int64(39), np.int64(48)]
Unexpected symbol 'const', skipping to state 18
State: 18	 Token: const	 Stack: [np.int64(30), np.int64(39), np.int64(48)]
State: 70	 Token: const	 Stack: [np.int64(30), np.int64(39), np.int64(48)]
ACCEPTING: const
State: 48	 Token: *	 Stack: [np.int64(30), np.int64(39)]
State: 10	 Toke

ValueError: ERROR